## Pipenv

In [ ]:
#gunicorn
#lightgbm==3.3.1
#scikit-learn==1.0.2
#xgboost==1.4.2
#catboost==1.0.3
#Flask
#pandas and numpy

In [2]:
import pandas as pd
import numpy as np
import json
import pickle

In [3]:
predictors = ['MPpg', 'PTSpg', 'FGpg', '3Ppg', 'FTpg', 'ORBpg', 'DRBpg', 'TRBpg',
       'ASTpg', 'STLpg', 'BLKpg', 'TOVpg', 'PFpg', 'Age', 'Year_index',
       'Team_index', 'FT', 'FT%', 'Games', 'MP', 'FG%']

In [4]:
with open('team_mapper.json') as json_file:
    team_mapper = json.load(json_file)

In [5]:
team_mapper

{'ATL': 0,
 'BOS': 1,
 'BRK': 2,
 'CHH': 3,
 'CHI': 4,
 'CLE': 5,
 'DAL': 6,
 'DEN': 7,
 'DET': 8,
 'GSW': 9,
 'HOU': 10,
 'IND': 11,
 'LAC': 12,
 'LAL': 13,
 'MEM': 14,
 'MIA': 15,
 'MIL': 16,
 'MIN': 17,
 'NOP': 18,
 'NYK': 19,
 'OKC': 20,
 'ORL': 21,
 'PHI': 22,
 'PHO': 23,
 'POR': 24,
 'SAC': 25,
 'SAS': 26,
 'TOR': 27,
 'UTA': 28,
 'WSH': 29}

In [6]:
models ={}
with open('rookie-models.bin', 'rb') as f_in:
    models['lr'],models['lgb'],models['xgb'],models['cats'],models['ada'],models['gbm'],models['rf'] = pickle.load(f_in)

C:\ProgramData\Anaconda3\lib\site-packages\dask\dataframe\utils.py:366: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
C:\ProgramData\Anaconda3\lib\site-packages\dask\dataframe\utils.py:366: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
C:\ProgramData\Anaconda3\lib\site-packages\dask\dataframe\utils.py:366: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\compat.py:31: FutureWarning: pandas.Int64Index is

In [7]:
models

{'lr': LogisticRegression(C=0.1, solver='liblinear'),
 'lgb': LGBMClassifier(colsample_bytree=0.9, learning_rate=0.008, max_depth=5,
                metric='binary_logloss', min_child_samples=10, n_estimators=5000,
                num_leaves=120, random_state=34, reg_alpha=5, reg_lambda=2,
                subsample=0.7, subsample_freq=2),
 'xgb': XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=0.9, gamma=0, gpu_id=-1,
               importance_type='gain', interaction_constraints='',
               learning_rate=0.01, max_delta_step=0, max_depth=5,
               min_child_weight=1, missing=nan, monotone_constraints='()',
               n_estimators=5000, n_jobs=-1, num_parallel_tree=1,
               random_state=34, reg_alpha=10, reg_lambda=1, scale_pos_weight=1,
               subsample=0.7, tree_method='exact', validate_parameters=1,
               verbosity=None),
 'cats': <catboost.core.CatBoostClassifier at

In [8]:
def predict_single(feature_dict_list,models,predictors):
    #feature_dict['Year_index'] = feature_dict['Year'] - 1979
    #feature_dict['Team_index'] = team_mapper[feature_dict['Team']]
    df = pd.DataFrame.from_dict(feature_dict_list)
    df['Year_index'] = df['Year'] - 1979
    df['Team_index'] = df['Team'].map(team_mapper)
    final_predictions = (
         (models['lr'].predict_proba(df[predictors])[:,1]) +
        (models['lgb'].predict_proba(df[predictors])[:,1]) +
        (models['xgb'].predict_proba(df[predictors])[:,1]) +
        (models['cats'].predict_proba(df[predictors])[:,1])+ 
        (models['ada'].predict_proba(df[predictors])[:,1])+
         (models['gbm'].predict_proba(df[predictors])[:,1])+
         (models['rf'].predict_proba(df[predictors])[:,1])
    )/7
    final_preds = (final_predictions>=0.52)*1
    return final_predictions,final_preds

In [10]:
l = [{'Unnamed: 0': 2458,
  'Player': 'Thanasis Antetokounmpo',
  'Team': 'NYK',
  'Year': 2016,
  'Conf': 'EAST',
  'Age': 23.0,
  'Career': 4.0,
  'Games': 2.0,
  'MP': 6.0,
  'PTS': 6.0,
  'FG': 3.0,
  'FGA': 4.0,
  'FG%': 0.75,
  '3P': 0.0,
  '3PA': 1.0,
  '3P%': 0.0,
  'FT': 0.0,
  'FTA': 0.0,
  'FT%': 0.0,
  'ORB': 0.0,
  'DRB': 1.0,
  'TRB': 1.0,
  'AST': 0.0,
  'STL': 0.0,
  'BLK': 0.0,
  'TOV': 0.0,
  'PF': 2.0,
  'MPpg': 3.0,
  'PTSpg': 3.0,
  'FGpg': 1.5,
  '3Ppg': 0.0,
  'FTpg': 0.0,
  'ORBpg': 0.0,
  'DRBpg': 0.5,
  'TRBpg': 0.5,
  'ASTpg': 0.0,
  'STLpg': 0.0,
  'BLKpg': 0.0,
  'TOVpg': 0.0,
  'PFpg': 1.0,
  'target>=2': 1,
  'target>=3': 1,
  'target>=4': 1,
  'target>=5': 0,
  'Team_index': 19,
  'Year_index': 37},
 {'Unnamed: 0': 2616,
  'Player': 'Ivica Zubac',
  'Team': 'LAL',
  'Year': 2016,
  'Conf': 'WEST',
  'Age': 19.0,
  'Career': 6.0,
  'Games': 38.0,
  'MP': 609.0,
  'PTS': 284.0,
  'FG': 126.0,
  'FGA': 238.0,
  'FG%': 0.529,
  '3P': 0.0,
  '3PA': 3.0,
  '3P%': 0.0,
  'FT': 32.0,
  'FTA': 49.0,
  'FT%': 0.653,
  'ORB': 41.0,
  'DRB': 118.0,
  'TRB': 159.0,
  'AST': 30.0,
  'STL': 14.0,
  'BLK': 33.0,
  'TOV': 30.0,
  'PF': 66.0,
  'MPpg': 16.03,
  'PTSpg': 7.5,
  'FGpg': 3.32,
  '3Ppg': 0.0,
  'FTpg': 0.84,
  'ORBpg': 1.08,
  'DRBpg': 3.11,
  'TRBpg': 4.2,
  'ASTpg': 0.8,
  'STLpg': 0.37,
  'BLKpg': 0.87,
  'TOVpg': 0.79,
  'PFpg': 1.74,
  'target>=2': 1,
  'target>=3': 1,
  'target>=4': 1,
  'target>=5': 1,
  'Team_index': 13,
  'Year_index': 37}]

In [11]:
predict_single(feature_dict_list=l,models=models,predictors=predictors)

(array([0.11916493, 0.87706843]), array([0, 1]))

In [12]:
predictors

['MPpg',
 'PTSpg',
 'FGpg',
 '3Ppg',
 'FTpg',
 'ORBpg',
 'DRBpg',
 'TRBpg',
 'ASTpg',
 'STLpg',
 'BLKpg',
 'TOVpg',
 'PFpg',
 'Age',
 'Year_index',
 'Team_index',
 'FT',
 'FT%',
 'Games',
 'MP',
 'FG%']

In [ ]:
['Year','Team','Age',]